# Random forest feature selection function

### Inspecting feature importance

In [102]:
## Import the random forest model.
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
    
def split_data(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12)
    return X_train, X_test, y_train, y_test

def train_rf_model(X_train,y_train):
    # Random forest model
    rf = RandomForestClassifier()

    # Fitting the model
    rf.fit(X_train, y_train)

    # Predicting values
    y_train_pred = rf.predict(X_train)
    y_test_pred = rf.predict(X_test)

    return rf, y_train_pred, y_test_pred

    

        

In [128]:
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
from mlxtend.evaluate import confusion_matrix


def plot_confusion_matrix(y_test,y_test_pred):
        # Transforming the predictions to the same shape for plotting with a confusion matrix
        try:
            if (y_test.shape[1] == 1):
                y_test = np.squeeze(np.asarray(y_test))
        except:
            print("y_test ok shape")
        try:
            if (y_test_pred.shape[1] == 1):
                y_test = np.squeeze(np.asarray(y_test_pred))
        except:
            print("y_test_pred ok shape")
              
                          
        # Plot
        cm = confusion_matrix(y_target=y_test,y_predicted=y_test_pred,binary=False)
        fig, ax = plot_confusion_matrix(conf_mat=cm,
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
        # Saving the figure to disk

In [129]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
data = load_wine()
#data.target[[10, 80, 140]]
#data.values

X = pd.DataFrame(data.data, columns = data.feature_names)
y = pd.DataFrame(data.target)
y = np.array(y)

X_train, X_test, y_train, y_test = split_data(X,y)

rf, y_train_pred, y_test_pred = train_rf_model(X_train, y_train)


plot_confusion_matrix(y_test,y_test_pred)



y_test_pred ok shape


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


TypeError: plot_confusion_matrix() got an unexpected keyword argument 'conf_mat'

## Feature imporatances

In [105]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
alcohol,0.205108
color_intensity,0.175121
flavanoids,0.165261
hue,0.128538
proline,0.110147
total_phenols,0.076486
od280/od315_of_diluted_wines,0.045250
proanthocyanins,0.040913
nonflavanoid_phenols,0.014627
alcalinity_of_ash,0.014297
